In [2]:
import numpy as np
import torch 
import gym 

/opt/conda/envs/DQN_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import cv2

In [15]:
import preprocessing_wrappers2
from preprocessing_wrappers2 import make_atari, wrap_deepmind

In [18]:
from train import make_env

In [11]:
env = gym.make('PongNoFrameskip-v4')

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [12]:
obs = env.reset()

In [13]:
obs.shape

(210, 160, 3)

TypeError: argument of type 'TimeLimit' is not iterable

In [19]:
wrapped_env = wrap_deepmind(make_atari(env))

TypeError: argument of type 'TimeLimit' is not iterable